# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [ ]:
!pip install -qU datasets pinecone-client==3.1.0 sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [ ]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True,
     trust_remote_code=True
).shuffle(seed=960)

wikipedia_snippets_streamed.py:   0%|          | 0.00/4.58k [00:00<?, ?B/s]

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [ ]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [ ]:
import itertools
# filter only documents with History as section_title - Replace None with your code
history_documents = (doc for doc in wiki_data if "history" in doc['section_title'].lower())

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [ ]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(itertools.islice(history_documents, total_doc_count), total=total_doc_count, desc="Processing documents"):
    docs.append({
        'article_title': d['article_title'],
        'section_title': d['section_title'],
        'passage_text': d['passage_text']
    })
    # increase the counter on every iteration
    counter += 1

Processing documents:   0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Synaphea spinulosa,Taxonomic history,"1768. Prior to this, the only known visit by E..."
1,Taupo District,History,was not until the 1950s that the region starte...
2,Sutarfeni,History & Western asian analogues,Sutarfeni History strand-like pheni were Phena...
3,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
4,Teufelsmoor,History & Situation today,"made to preserve the original landscape, altho..."


In [ ]:
# Save to a pickle file (more efficient for large datasets)
df.to_pickle("history_passages.pkl")

# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY")

# configure client
pc = Pinecone(api_key=api_key)

Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [ ]:
index_name = "abstractive-question-answering" #give your index a meaningful name
dimension = 768

In [ ]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    print(f"Creating index '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=spec
    )
else:
    print(f"Index '{index_name}' already exists.")

# Wait for the index to initialize
print("Waiting for the index to initialize...")
time.sleep(10)

# Connect to the index
index = pc.Index(index_name)

Index 'abstractive-question-answering' already exists.
Waiting for the index to initialize...


# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device) #load the retriever model from HuggingFace. Use the flax-sentence-embeddings/all_datasets_v3_mpnet-base model
retriever

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [54]:
# we will use batches of 64
batch_size = 64

#You will create embedding for the passage_text variable and be use to include the meta data in each batch
for i in tqdm(range(0, len(df), batch_size), desc="Upserting vectors"):
    end = min(i + batch_size, len(df))
    batch = df.iloc[i:end]
    embeddings = retriever.encode(batch['passage_text'].tolist(), show_progress_bar=False)
    to_upsert = [
        (str(idx + i), embedding.tolist(), {
            "article_title": row.article_title,
            "section_title": row.section_title,
            "passage_text": row.passage_text
        })
        for idx, (embedding, row) in enumerate(zip(embeddings, batch.itertuples(index=False)))
    ]
    index.upsert(vectors=to_upsert)

Upserting vectors:   0%|          | 0/782 [00:00<?, ?it/s]

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [55]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [64]:
def query_pinecone(query, top_k=10):
    # Generate embeddings for the query
    xq = retriever.encode([query], show_progress_bar=False)

    # Convert the embedding to a Python list
    xq = xq[0].tolist()

    # Search Pinecone index for context passages
    xc = index.query(
        vector=xq,
        top_k=top_k,
        namespace='',
        include_metadata=True
    )
    return xc['matches']

In [97]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    instructions = "You are an expectect at answering user questions. Due not repeat the same sentences that you have already said. Write your responce in scientific style. Be precise and accurate in your responce"
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"instructions: {instructions} question: {query} context: {context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [65]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

[{'id': '4506',
  'metadata': {'article_title': 'Electric power system',
               'passage_text': 'Electric power system History In 1881, two '
                               "electricians built the world's first power "
                               'system at Godalming in England. It was powered '
                               'by two waterwheels and produced an alternating '
                               'current that in turn supplied seven Siemens arc '
                               'lamps at 250 volts and 34 incandescent lamps at '
                               '40 volts. However, supply to the lamps was '
                               'intermittent and in 1882 Thomas Edison and his '
                               'company, The Edison Electric Light Company, '
                               'developed the first steam-powered electric '
                               'power station on Pearl Street in New York City. '
                               'The Pearl Street Sta

In [66]:
from pprint import pprint

In [68]:
# format the query in the form generator expects the input
query = format_query(query, result)
pprint(query)

('question: when was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


The output looks great. Now let's write a function to generate answers.

In [101]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40,repetition_penalty=1.5)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [70]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [72]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context)
generate_answer(query)

('The first wireless message was sent in 1866 by Mahlon Loomis using a kite to '
 'transmit an electrical signal through the atmosphere between two 600 foot '
 'wires held aloft by kites')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [79]:
for doc in context:
    print(doc["metadata"]["passage_text"], end='\n---\n')

to establish with certainty which diseases jumped from other animals to humans, but there is increasing evidence from DNA and RNA sequencing, that measles, smallpox, influenza, HIV, and diphtheria came to humans this way. Various forms of the common cold and tuberculosis also are adaptations of strains originating in other species.
Zoonoses are of interest because they are often previously unrecognized diseases or have increased virulence in populations lacking immunity. The West Nile virus appeared in the United States in 1999 in the New York City area, and moved through the country in the summer of 2002, causing much distress. Bubonic
---
plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease.
A major factor contributing to the appearance of new zoonotic pathogens in human populations is increased contact between humans and wildlife. This can be caused either by encroachment of human activity into wilderness areas or by movement of wild ani

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [76]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

('COVID-19 is a zoonotic disease, which means that it is a virus that is '
 'transmitted from one animal to another. It is not a virus that can be '
 'transmitted from person')


In [78]:
for doc in context:
    print(doc["metadata"]["passage_text"], end='\n---\n')

to establish with certainty which diseases jumped from other animals to humans, but there is increasing evidence from DNA and RNA sequencing, that measles, smallpox, influenza, HIV, and diphtheria came to humans this way. Various forms of the common cold and tuberculosis also are adaptations of strains originating in other species.
Zoonoses are of interest because they are often previously unrecognized diseases or have increased virulence in populations lacking immunity. The West Nile virus appeared in the United States in 1999 in the New York City area, and moved through the country in the summer of 2002, causing much distress. Bubonic
---
plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease.
A major factor contributing to the appearance of new zoonotic pathogens in human populations is increased contact between humans and wildlife. This can be caused either by encroachment of human activity into wilderness areas or by movement of wild ani

Let’s finish with a final few questions.

In [81]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context)
generate_answer(query)

('The War of Currents was a series of events in the early 20th century that '
 'pitted Edison and Westinghouse against each other over the use of '
 'alternating current (AC) and direct current')


In [82]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context)
generate_answer(query)

('The first person to walk on the Moon was Giovanni Battista Fortuna, who '
 'walked on the Moon in 1969. He was the first man to walk on the Moon, and he '
 'was')


In [83]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions

In [84]:
query = "what is the most expensive Museum in the world?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

('The Louvre in Paris is the most expensive museum in the world. The Louvre is '
 'a collection of art from all over the world, and the Louvre is the most '
 'expensive museum in')


In [85]:
query = "who was the first person to discover the light bulb?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context)
generate_answer(query)

('The first person to discover the light bulb was Thomas Edison, who '
 'demonstrated it at an exhibition in New York City on October 29, 1878. The '
 'Edison Electric-Light Company of New York')


In [102]:
query = "what was the cold war?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context)
generate_answer(query)

('The Cold War was a period of time when the United States and the Soviet '
 'Union were at war with each other. The US and the Soviet Union had been at '
 'war since the 1930s,')


In [103]:
query = "why is it cold in the North pole?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context)
generate_answer(query)

("The North Pole has a lot of ice, and it's very cold there. It's not as cold "
 "as the rest of the world, but it's still cold.")


In [111]:
query = "What was the average life expectancy in the USA?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context)
generate_answer(query)

('The average life expectancy in the US was 76.1 years for a male and 81.4 '
 'years for a female in the 1950 census. The difference between the two is '
 'that the median age')
